## Document Data Capture 
#### Captura de Dados de Documentos

### The objective of this project is — through an agent — to perform the following tasks:

* Extract tables from documents;
* Retrieve documents from a Mongo database;
* Insert tables extracted from documents into an SQLite database;
* Query specific information from tables stored in an SQLite database.

---

### O objetivo deste projeto é — através de um agente — desempenhar as seguintes tarefas:

* Realizar a extração de tabelas de documentos;
* Buscar documentos do bancco de dados Mongo;
* Inserir as tabelas extraídas de documentos em um banco de dados SQLite;
* Consultar informações específicas em tabelas presentes em um banco de dados SQLite.

#### Installation of Libraries
Instalação das Bibliotecas

In [15]:
import os
import base64
import sqlite3
import logging
import pymongo
from langchain.tools import tool
from dataclasses import dataclass
from typing import Callable, List
from bson.objectid import ObjectId
from langgraph.types import Command
from langgraph.runtime import Runtime
from pymongo.synchronous.database import Database
from pymongo.synchronous.collection import Collection
from langchain.agents import AgentState, create_agent
from langchain.tools.tool_node import ToolCallRequest
from langgraph.checkpoint.memory import InMemorySaver
from pymongo.synchronous.mongo_client import MongoClient
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.messages import RemoveMessage, ToolMessage
from langchain.agents.structured_output import ToolStrategy
from langchain.agents.middleware import wrap_tool_call, after_model

#### Defining Environment Variables and Constants
Definção de Variáveis de Ambiente e Constantes

In [16]:
# API key name
# Nome da chave API
os.environ["GOOGLE_API_KEY"] = "F9lsEaP1k&Y"

# Mongo database name
# Nome do banco de dados Mongo
os.environ["DATABASE_NAME"] = "mongo_database"

# Mongo database collection name
# Nome da coleção do banco de dados Mongo
os.environ["COLLECTION_NAME"] = "mongo_collection"

# Mongo database URI
# URI do banco de dados Mongo
os.environ["MONGO_URI"] = "mongodb://localhost:27017/"

# Path to the file used as the SQLite database
# Caminho do arquivo utilizado como banco de dados SQLite
os.environ["DATABASE_PATH"] = "./essay/databases/langchain_database.db"

In [17]:
# Path to the directory where the PDF files are stored
# Caminho do diretório onde estão armazenados os arquivos em formato PDF
DIR_PATH = "../essay/documents/"

# Agent roles
# Funções do agente
SYSTEM_PROMPT = """

You are an agent specialized in extracting structured data from documents and performing actions on an SQLite database, such as creating tables, inserting information into them, and querying data from tables that already exist.

---

# Available Tools

1. Tool 'create_table_in_the_database': use this tool to create a table in the database based on the data extracted from a document.

Rules:
- You must define the table name according to the context of the data extracted from the document;
- You must define the table column names according to the context of the data extracted from the document;
- Use this tool when the user requests the creation of a table in the database containing the information extracted from the document;
- You must create the SQL command for table creation for an SQLite database, compatible with the syntax of Python's 'sqlite3' library. The SQL command must be written between single quotes.

Inputs:
- sql: SQL command for creating the table (String);

Example:
CREATE TABLE movie(title, year, score)

2. Tool 'inserts_information_into_the_table': use this tool immediately after the 'create_table_in_the_database' tool. The purpose of this tool is to insert information extracted from a document into a newly created table in the database.

Rules:
- Use this tool when the user requests inserting information extracted from a document into the database;
- You must create the SQL command for inserting data into the table previously created in the SQLite database. The SQL command must be written between triple quotes, as a docstring;
- You must respect the order of insertion of the information extracted from the document into the newly created table, maintaining consistency between each column and the data being inserted into it.

Inputs:
- sql: SQL command for inserting information into the table (String in docstring format);

Example:
INSERT INTO movie VALUES
   ('Monty Python and the Holy Grail', 1975, 8.2),
   ('And Now for Something Completely Different', 1971, 7.5)

3. Tool 'query_data_in_table': use this tool to query data from a table that already exists in the database.

Rules:
- The user must inform which data they want to extract from the table in the database;
- The user must inform which table they want to query, if it is not possible to deduce it from the context of the conversation;
- The SQL query command for the SQLite database must be constructed based on the information the user wants to retrieve from the table;
- You must create the SQL query command for the SQLite database, compatible with the syntax of Python's 'sqlite3' library. The SQL command must be written between single quotes.

Inputs:
- sql: SQL query command (String);

Examples:
SELECT score FROM movie  
SELECT title FROM movie WHERE title='back to the future'

---

# Available Middleware

1. Middleware 'wrap_tool_call': use this middleware to display the monitoring of each tool in use.
2. Middleware 'delete_old_messages': use this middleware to delete the oldest messages from a conversation's message history.

---

# Agent Responsibilities

1. Extract data from documents as requested by the user;  
3. Query information from tables already existing in the database, according to the user's request;
2. Create tables in the database and insert data into them, always following the order 'create → insert'.  

---

# Additional Rules
 
1. If any user request or message doesn't make sense, return a default response;
2. Use the middleware in each tool call to enable monitoring of the tools being used;
3. The data extracted from a document should be presented to the user in an organized manner, indicating each column and its respective values;
4. User requests for creating tables or inserting random information into the database — information not originating from a document — must not be fulfilled;
5. Information extracted from a document must be inserted immediately after the table is created in the database. For this reason, after the user requests inserting data from a document into the database, the use of the 'create_table_in_the_database' tool must precede the use of the 'inserts_information_into_the_table' tool.

"""

#### Definition of the Tools Used by the Agent
Definição das Ferramentas Utilizadas Pelo Agente

In [64]:
@tool(name_or_callable="create_table_in_the_database", description="This tool creates a table in the database. Use it after extracting structured data from documents.")
def create_table_in_the_database(sql: str) -> bool:
    """Creates the extracted table from a document in the database.
    
       Args: 
           sql: SQL command to create a table in the database.
    """
    connection = sqlite3.connect(os.getenv("DATABASE_PATH"))

    cursor = connection.cursor()
    cursor.execute(sql)
    connection.close()
    
    return True

In [65]:
@tool(name_or_callable="inserts_information_into_the_table", description="This tool inserts information into a previously created table in the database. Use it to add data to a pre-existing table in the database.")
def inserts_information_into_the_table(sql: str) -> bool:
    """Inserts information into a previously created table in the database.
    
       Args: 
           sql: SQL command to insert information into a previously created table in the database.
    """
    connection = sqlite3.connect(os.getenv("DATABASE_PATH"))

    cursor = connection.cursor()
    cursor.execute(sql)
    connection.commit()
    connection.close()
    
    return True

In [66]:
@tool(name_or_callable="query_data_in_table", description="This tool queries data in pre-existing tables in the database. Use it to extract specific data from tables present in the database.")
def query_data_in_table(sql: str) -> str:
    """Querying information in tables in the database.
      
       Args:
           sql: SQL command to extract specific information from tables in the database, according to user requests.
    """
    connection = sqlite3.connect(os.getenv("DATABASE_PATH"))

    cursor = connection.cursor()
    raw_response = cursor.execute(sql)
    refined_response = raw_response.fetchall()
    connection.close()

    return refined_response

#### Handling Exceptions as Tools are Called by the Agent
Tratamento de Exceções ao Passo que as Ferramentas São Chamadas Pelo Agente

In [67]:
@wrap_tool_call
def monitor_tool(
    request: ToolCallRequest,
    handler: Callable[[ToolCallRequest], ToolMessage | Command],
) -> ToolMessage | Command:
    
    logging.info(f"Running a tool '{request.tool_call['name']}' with the following arguments: {request.tool_call['args']}")
    
    try:
        result = handler(request)
        return result
    except Exception as error:
        raise Exception(f"The tool could not be executed: {error}")

#### Removing Part of the Message History From a Dialogue with the Agent
Remoção de Parte do Histórico de Mensagens de um Diálogo com o Agente

In [68]:
@after_model
def delete_old_messages(state: AgentState, runtime: Runtime) -> dict | None:
    messages = state["messages"]
    if len(messages) > 10:
        return {"messages": [RemoveMessage(id=message.id) for message in messages[:3]]}
    return None

#### Definition of a Standard Response Provided by the Agent
Definição de Uma Resposta Padrão Concedida Pelo Agente

In [69]:
@dataclass
class ResponseFormat:
    punny_response: str
    default_response: str = "Your message is outside the expected standard."

#### Definition of the Model Used by the Agent
Definição do Modelo Utilizado Pelo Agente

In [70]:
model = ChatGoogleGenerativeAI(
    timeout=60,
    temperature=0.5,
    max_tokens=125000,
    model="gemini-2.5-flash"
)

#### Definition of Agent Memory
Definição da Memória do Agente

In [71]:
checkpointer = InMemorySaver()

#### Agent Definition
Definição do Agente

In [72]:
agent = create_agent(
    model=model,
    checkpointer=checkpointer,
    system_prompt=SYSTEM_PROMPT,
    middleware=[monitor_tool, delete_old_messages],
    response_format = ToolStrategy(ResponseFormat),
    tools = [create_table_in_the_database, inserts_information_into_the_table, query_data_in_table]
)

#### Dialog ID Definition
Definição do ID do Diálogo

In [73]:
config = {"configurable": {"thread_id": "1"}}

#### Converting PDF Files to Base64
Conversão de Arquivo com Extensão PDF para Base 64

In [16]:
def convert_pdf_to_base64(path_file: str) -> str:
    with open (path_file, "rb") as file:
        data = file.read()
    
    encoded_data = base64.b64encode(data)
    encoded_data = encoded_data.decode("utf-8")

    return encoded_data

### Step 1
Passo 1

#### Definition of Messages Sent to the Agent 
Definição das Mensagens Enviadas ao Agente


In [20]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Extract the table containing the title 'Simple Table' from this document."},
            {"type": "file", "base64": convert_pdf_to_base64("./documents/1.pdf"), "mime_type": "application/pdf"}
        ]
    }
]

#### Agent Summon
Invocação do Agente

In [21]:
response = agent.invoke(
    config=config,
    input={"messages": messages}
)

#### Agent Response Display
Exibição da Resposta do Agente

In [22]:
print(response["structured_response"])

ResponseFormat(punny_response="I've extracted the 'Simple Table' for you! Here's what I found:\n\n**Information 1**: A, B, C\n**Information 2**: Cat, Dog, Shark\n**Information 3**: 4.7, 2.8, 3.8\n**Information 4**: 5, 6, 8", default_response='Your message is outside the expected standard.')


### Step 2
Passo 2

#### Definition of Messages Sent to the Agent
Definição das Mensagens Enviadas ao Agente

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Place this information that you just extracted from the document into the database."}
        ]
    }
]

#### Agent Summon
Invocação do Agente

In [47]:
response = agent.invoke(
    config=config,
    input={"messages": messages}
)

#### Agent Response Display
Exibição da Resposta do Agente

In [48]:
print(response["structured_response"])

ResponseFormat(punny_response="The information has been successfully placed into the database in a table named 'simple_table'.", default_response='Your message is outside the expected standard.')


### Step 3
Passo 3

#### Definition of Messages Sent to the Agent
Definição das Mensagens Enviadas ao Agente

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Extract the table containing the title 'Basic Table' from this document."},
            {"type": "file", "base64": convert_pdf_to_base64("./documents/2.pdf"), "mime_type": "application/pdf"}
        ]
    }
]

#### Agent Summon
Invocação do Agente

In [25]:
response = agent.invoke(
    config=config,
    input={"messages": messages}
)

#### Agent Response Display
Exibição da Resposta do Agente

In [26]:
print(response["structured_response"])

ResponseFormat(punny_response="I've extracted the 'Basic Table' from the document. Here's the data:\n\nColumn 1: Category\nColumn 2: Value\n\n- Animals: 7\n- Instruments: 8\n- Birds: 9\n- Flowers: 17\n\nWould you like me to create a table in the database with this information?", default_response='Your message is outside the expected standard.')


### Step 4
Passo 4

#### Definition of Messages Sent to the Agent
Definição das Mensagens Enviadas ao Agente

In [25]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Place this information that you just extracted from the document into the database."}
        ]
    }
]

#### Agent Summon
Invocação do Agente

In [26]:
response = agent.invoke(
    config=config,
    input={"messages": messages}
)

#### Agent Response Display
Exibição da Resposta do Agente

In [27]:
print(response["structured_response"])

ResponseFormat(punny_response="The information from the 'Basic Table' has been successfully placed into the database!", default_response='Your message is outside the expected standard.')


### Step 5
Passo 5

#### Definition of Messages Sent to the Agent
Definição das Mensagens Enviadas ao Agente

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Now, I want you to run a query on the database. I want you to extract the information from the 'Information 1' column of the 'Simple Table' table created earlier in the database and display the queried information."}
        ]
    }
]

#### Agent Summon
Invocação do Agente

In [32]:
response = agent.invoke(
    config=config,
    input={"messages": messages}
)

#### Agent Response Display
Exibição da Resposta do Agente

In [ ]:
print(response["structured_response"])

ResponseFormat(punny_response="Here is the information from the 'Information 1' column of the 'Simple Table':\n\n- A\n- B\n- C", default_response='Your message is outside the expected standard.')


#### Removal of All Dialogue with the Agent
Remoção de Todo o Diálogo com o Agente

In [ ]:
checkpointer.delete_thread(thread_id="1")

#### Creating a Connection to the Mongo Database
Criação de Conexão com o Banco de Dados Mongo

In [19]:
def create_connection() -> MongoClient:
    mongo_client = pymongo.MongoClient(os.getenv("MONGO_URI") + os.getenv("DATABASE_NAME"))

    try:
        mongo_client.admin.command("ping")
        return mongo_client
    except Exception as error:
        raise Exception(f"Unable to establish a connection: {error}")

mongo_client = create_connection()

#### Mongo Database Generation
Geração do Banco de Dados Mongo

In [20]:
def create_database(mongo_client: MongoClient) -> Database:
    mongo_database = getattr(mongo_client, os.getenv("DATABASE_NAME"))
    return mongo_database

mongo_database = create_database(mongo_client)

#### Creating a Collection in a Mongo Database
Criação de Uma Coleção no Banco de Dados Mongo

In [21]:
def create_collection(mongo_database: Database) -> Collection:
    mongo_collection = getattr(mongo_database, os.getenv("COLLECTION_NAME"))
    return mongo_collection

mongo_collection = create_collection(mongo_database)

#### Inserting Files into a Mongo Database
Inserção de Arquivos no Banco de Dados Mongo

In [17]:
def insert_file_into_the_database(mongo_collection: Collection) -> bool:
    file_list = []

    for file_path in os.listdir(DIR_PATH):
        file_list.append({"file": convert_pdf_to_base64(DIR_PATH + file_path), "name": file_path})
    
    acknowledged = getattr(mongo_collection.insert_many(file_list), "acknowledged")
    return acknowledged

acknowledged = insert_file_into_the_database(mongo_collection)

#### Searching for Files in a Mongo Database
Busca de Arquivos no Banco de Dados Mongo

In [25]:
def search_for_all_files(mongo_collection: Collection) -> List[dict]:
    file_data = []

    for data in mongo_collection.find():
        file_data.append(data)
    
    return file_data

file_data = search_for_all_files(mongo_collection)

In [ ]:
def search_for_specific_file(mongo_collection: Collection, id: str) -> dict:
    file_data = mongo_collection.find_one({"_id": ObjectId(id)})
    return file_data

file_data = search_for_specific_file(mongo_collection, "697cae76563b6e14cf9eb3a3")